## Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving updated_dataset_places_to_eat_in_the_jogja_region.csv to updated_dataset_places_to_eat_in_the_jogja_region (1).csv


## Load dataset

In [ ]:
# Load dataset
file_path = 'updated_dataset_places_to_eat_in_the_jogja_region.csv'
data = pd.read_csv(file_path)

# Ubah kolom 'Harga Rata-Rata Makanan di Toko (Rp)' menjadi numerik tanpa titik
data['Harga Rata-Rata Makanan di Toko (Rp)'] = data['Harga Rata-Rata Makanan di Toko (Rp)'].astype(str).replace('.', '').astype(float)

# Pilih fitur yang akan digunakan untuk model
selected_features = ['Harga Rata-Rata Makanan di Toko (Rp)', 'Entertainment', 'Keramaian Restoran']

In [ ]:
# Ekstrak fitur dan target
X = data[selected_features]
y = data['Rating Toko'].values  # Asumsikan ada kolom 'Rating Toko'

In [ ]:
# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Inisiasi model regresi

In [ ]:
# Inisialisasi model
def create_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dense(1))  # Output layer untuk regresi
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='mean_squared_error')
    return model

# Fungsi untuk memprediksi rating berdasarkan fitur restoran
def predict_rating(features, scaler, model_path='restaurant_rating_model.h5'):
    features_scaled = scaler.transform([features])
    model = load_model(model_path)
    predicted_rating = model.predict(features_scaled)
    return predicted_rating[0][0]

## Evaluasi dengan Cross-Validation

In [ ]:
# Definisikan K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# List untuk menyimpan skor MSE dari setiap fold
mse_scores = []

# Lakukan cross-validation
fold_idx = 0
for train_idx, val_idx in kfold.split(X_scaled):
    fold_idx += 1
    print(f'Fold {fold_idx}')

    # Bagi data menjadi data latih dan validasi
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Inisialisasi ulang model untuk setiap fold
    model = create_model()

    # Latih model
    history = model.fit(X_train, y_train, epochs=50, verbose=0, validation_data=(X_val, y_val))

    # Evaluasi model pada data validasi
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    mse_scores.append(mse)
    print(f'  MSE: {mse}')

Fold 1
2/2 [==============================] - 0s 7ms/step
  MSE: 0.30550504809762186
Fold 2
2/2 [==============================] - 0s 7ms/step
  MSE: 0.49375466576249843
Fold 3
2/2 [==============================] - 0s 8ms/step
  MSE: 0.459006381461024
Fold 4
2/2 [==============================] - 0s 6ms/step
  MSE: 0.4424343214610371
Fold 5


2/2 [==============================] - 0s 7ms/step
  MSE: 1.3867594776787053


## Hasil MSE dari Cross-validation

In [ ]:
# Hasil rata-rata MSE dari cross-validation
mean_mse = sum(mse_scores) / len(mse_scores)
print(f'Rata-rata MSE dari 5-fold cross-validation: {mean_mse}')

Rata-rata MSE dari 5-fold cross-validation: 0.6174919788921773


In [ ]:
# Simpan model
model.save('restaurant_rating_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Contoh Penggunaan Model

In [ ]:
# Contoh penggunaan: Memprediksi rating restoran baru
new_restaurant_features = [30000, 1, 2]  # [Harga, Entertainment, Keramaian]
predicted_rating = predict_rating(new_restaurant_features, scaler)
clipped_rating = np.clip(predicted_rating, 1, 5)  # Batasi rating antara 1 dan 5
print(f'Prediksi Rating Toko untuk restoran baru: {clipped_rating}')

# Menampilkan restoran dengan harga mendekati atau sama dengan prediksi
predicted_price = new_restaurant_features[0]  # Ambil harga yang diprediksi

restaurants_with_predicted_price = data[
    (data['Harga Rata-Rata Makanan di Toko (Rp)'] >= predicted_price * 0.9) &
    (data['Harga Rata-Rata Makanan di Toko (Rp)'] <= predicted_price * 1.1)
]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 55ms/step
Prediksi Rating Toko untuk restoran baru: 4.272122383117676


In [ ]:
print("\nRestoran dengan harga mendekati atau sama dengan prediksi:")
# print(restaurants_with_predicted_price[['Nama Restoran', 'Harga Rata-Rata Makanan di Toko (Rp)', 'Rating Toko']])
display(restaurants_with_predicted_price[['Nama Restoran', 'Harga Rata-Rata Makanan di Toko (Rp)', 'Rating Toko']])


Restoran dengan harga mendekati atau sama dengan prediksi:


,Nama Restoran,Harga Rata-Rata Makanan di Toko (Rp),Rating Toko
13,"Sushi Battousai, Suryodiningratan",32000.0,4.5
18,"Rustik Bistro & Bar, Hotel Harper Malioboro Yo...",32000.0,3.8
19,"Gyudel Bowl & Nasi Goreng, Depok",29000.0,4.7
25,"Mamam Food, Pringgolayan Banguntapan",27000.0,4.7
26,"Yamie Panda, Cokroaminoto",28000.0,4.7
30,"Double O Resto, Tegalrejo",30000.0,4.6
32,Salad Sehat Fd,30000.0,4.7
34,"GYU KATSU, CONDONGCATUR",32000.0,4.7
55,"Warung Ramen N Katsu, Chapter 1 Menteri Supeno",30000.0,4.6
61,Penyetan Ayam Bakar Dan Suki Tomyam Yss Food,32000.0,4.8


In [ ]:
!pip install tensorflowjs
!tensorflowjs_converter --input_format keras restaurant_rating_model.h5 tfjs_model


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 927.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Succ

2024-06-19 15:46:21.613969: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!ls tfjs_model


group1-shard1of1.bin  model.json


In [ ]:
!zip -r tfjs_model.zip tfjs_model
from google.colab import files
files.download('tfjs_model.zip')


  adding: tfjs_model/ (stored 0%)
  adding: tfjs_model/model.json (deflated 70%)
  adding: tfjs_model/group1-shard1of1.bin (deflated 3%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained Keras model
model = load_model('restaurant_rating_model.h5')

# Save the model in the TensorFlow SavedModel format
saved_model_path = 'saved_model'
model.save(saved_model_path, save_format='tf')

# Verify the saved model
print("Model saved in the following directory structure:")
!ls {saved_model_path}
!ls {saved_model_path}/assets
!ls {saved_model_path}/variables


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7cb21ec9fc70>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7cb21ec9fc70>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model saved in the following directory structure:
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables
variables.data-00000-of-00001  variables.index


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained Keras model
model = load_model('restaurant_rating_model.h5')

# Save the model in the TensorFlow SavedModel format
saved_model_path = 'saved_model'
model.save(saved_model_path, save_format='tf')

# Verify the saved model directory structure
import os

print("Model saved in the following directory structure:")
print(os.listdir(saved_model_path))
print(os.listdir(os.path.join(saved_model_path, 'assets')))
print(os.listdir(os.path.join(saved_model_path, 'variables')))

# Compress the saved model directory for easy downloading
!zip -r saved_model.zip saved_model

# Download the compressed model
from google.colab import files
files.download('saved_model.zip')


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7cb21ea31e10>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7cb21ea31e10>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model saved in the following directory structure:
['assets', 'saved_model.pb', 'variables', 'fingerprint.pb', 'keras_metadata.pb']
[]
['variables.index', 'variables.data-00000-of-00001']
  adding: saved_model/ (stored 0%)
  adding: saved_model/assets/ (stored 0%)
  adding: saved_model/saved_model.pb (deflated 86%)
  adding: saved_model/variables/ (stored 0%)
  adding: saved_model/variables/variables.index (deflated 55%)
  adding: saved_model/varia

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>